<a href="https://colab.research.google.com/github/neculaluana/Twitter-emotion-analysis/blob/main/emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import re
import torch
import plotly.express as px

from google.colab import drive
from datasets import Dataset, DatasetDict, Features, Value, ClassLabel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DataCollatorWithPadding
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/neculaluana/Twitter-emotion-analysis/main/input/training.csv?token=GHSAT0AAAAAACBUYC47QMRBYGX5L5L7H6FGZCBAPEA')
df_test = pd.read_csv('https://raw.githubusercontent.com/neculaluana/Twitter-emotion-analysis/main/input/test.csv?token=GHSAT0AAAAAACBUYC477ZGWJ43J7OLYWPVGZCBAM7Q')
df_valid = pd.read_csv('https://raw.githubusercontent.com/neculaluana/Twitter-emotion-analysis/main/input/validation.csv?token=GHSAT0AAAAAACBUYC47FCWVRUS45ZFJAQOYZCBAQDA')

In [ ]:
def clean_tweet(tweet):

  tweet = re.sub(r'https?://[^ ]+', '', str(tweet))      #removes links
  tweet = re.sub(r'@[^ ]+', '', str(tweet))              #removes mentions
  tweet = re.sub(r'#', '', str(tweet))                   #removes hashtag symbol
  tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', str(tweet)) #removes repeated characters ex: heeeeeeey
  tweet = re.sub(r'[^A-Za-z ]', '', str(tweet))          #removes unwanted characters and punctuation
  tweet = re.sub(r' 0 ', 'zero', str(tweet))             #transforms 0 to zero (it can influence emotion)
  tweet = tweet.lower()                                  #lower-casing
  return tweet

In [ ]:
df = pd.concat([df_train, df_valid, df_test], ignore_index=True, sort=False)
df["text"]=df["text"].apply(lambda text: clean_tweet(text))
print(df)

In [ ]:
emotion_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
custom_features = Features({
    'text': Value(dtype='string'),
    'label': ClassLabel(names=emotion_names)
})

"""emotions_full = DatasetDict({
    "train": Dataset.from_pandas(df_train,features=custom_features),
    "test": Dataset.from_pandas(df_test,features=custom_features),
    "validation": Dataset.from_pandas(df_valid,features=custom_features)
    })

"""
emotions_full_dataset = Dataset.from_pandas(df, features=custom_features)
data_column=emotions_full_dataset ["text"]
label_column=emotions_full_dataset ["label"]


X_train, X_val, y_train, y_val = train_test_split(data_column, label_column, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

dataset = DatasetDict({"train": Dataset.from_dict({"text": X_train, "label": y_train}),
                        "validation": Dataset.from_dict({"text": X_val, "label": y_val}),
                        "test": Dataset.from_dict({"text": X_test, "label": y_test})})

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize, batched=True, batch_size=None)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
drive.mount('/content/gdrive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6).to(device)